In [1]:
# ! pip install tweepy
# ! pip install random-word
import os
from tqdm import tqdm
import tweepy
import pandas as pd
from random_word import RandomWords

In [2]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAE1kaAEAAAAA5eq%2B9MQY1ZxmqsqI8hPhFTgXy%2BI%3DDiHeboF9HqPkABqkGhzUTegzzsJGvXvZNd0Op3kVi7TvZlhTMI" 
consumer_key = "kSuunUrf10tDwGT1PhNDWj4HY"
consumer_secret = "Mg7Apz65TCxZqPhlnlQV8aBMuRhoDtFOljqbc3uoHp3F3uMzhh"
access_token = "1239561788082933760-bUTmKDXfsLoBKlVBb3twda1jUkxQWK"
access_token_secret = "jBxSnUcGx7l4OIfaGkIGmZQ6v3DAdq5tZyxmhP7J7w2dQ"

# Get Source Data 

In [3]:
client = tweepy.Client(
    bearer_token,
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

In [5]:
# generate random word for queries
n = 1000 # min number of queries
queries = set()

r = RandomWords()
while len(queries) < n:
    print(f"Current queries length: {len(queries)}")
    temp = r.get_random_words(includePartOfSpeech="noun,verb",
                              minCorpusCount = 100000)
    try:
        queries = set().union(queries, temp)
    except:
        continue
    
queries = list(queries)
print(f"Get a query poll of length {len(queries)}. Sample queries {queries[:10]}")

Current queries length: 0
Current queries length: 15
Current queries length: 23
Current queries length: 33
Current queries length: 45
Current queries length: 64
Current queries length: 74
Current queries length: 87
Current queries length: 98
Current queries length: 116
Current queries length: 116
Current queries length: 125
Current queries length: 142
Current queries length: 157
Current queries length: 164
Current queries length: 174
Current queries length: 186
Current queries length: 196
Current queries length: 203
Current queries length: 210
Current queries length: 225
Current queries length: 231
Current queries length: 243
Current queries length: 254
Current queries length: 261
Current queries length: 270
Current queries length: 282
Current queries length: 292
Current queries length: 303
Current queries length: 314
Current queries length: 324
Current queries length: 334
Current queries length: 341
Current queries length: 344
Current queries length: 353
Current queries length: 357
Cu

In [13]:
# create a dataframe from search result
def CreateDataFrame(tweets):
    id_list = [tweet.id for tweet in tweets[0]]
    text_list = [tweet.text for tweet in tweets[0]]
    non_hashtags_text_list = []
    author_id_list = [tweet.author_id for tweet in tweets[0]]
    hashtags_list = []
    for tweet in tweets[0]:
        try:
            if tweet.entities.get("hashtags"):
                hashtags_list.append([t["tag"] for t in tweet.entities.get("hashtags")])
                nontag_text = tweet.text
                for t in tweet.entities.get("hashtags"):
                    nontag_text = nontag_text.replace("#"+t["tag"], "<UNKTAG>")
                non_hashtags_text_list.append(nontag_text)
            else:
                hashtags_list.append([])
                non_hashtags_text_list.append(tweet.text)
        except:
            hashtags_list.append([])
            non_hashtags_text_list.append(tweet.text)
    df = pd.DataFrame({
        "id": id_list,
        "text": text_list,
        "non_hashtags_text": non_hashtags_text_list,
        "author_id": author_id_list,
        "hashtags": hashtags_list
    })
    return df

In [7]:
# get test dataframe from one query
tweets = client.search_recent_tweets(query=queries[0], 
                                     tweet_fields=['author_id', 'entities'],
                                     max_results=100)
test_df = CreateDataFrame(tweets)
test_df.head()

,id,text,non_hashtags_text,author_id,hashtags
0,1512575901279608835,RT @PopBase: Las Vegas turned purple in honor ...,RT @PopBase: Las Vegas turned purple in honor ...,1079167585856282624,[]
1,1512575900684107783,He continues to recruit as much as possible an...,He continues to recruit as much as possible an...,86140975,[]
2,1512575894950457356,@Gianni2225 Funny how so many people proclaime...,@Gianni2225 Funny how so many people proclaime...,991448824647479296,[]
3,1512575894841434122,RT @LCSOfficial: From 5-time Challenger to the...,RT @<UNKTAG>Official: From 5-time Challenger t...,2297906262,[LCS]
4,1512575893796966406,RT @BTSMerchUpdates: ALLEGIANT STADIUM FOOD\n\...,RT @BTSMerchUpdates: ALLEGIANT STADIUM FOOD\n\...,894038577008574464,[]


In [37]:
def GetTaggedTweet(tweet_df):
    tagged_df = tweet_df[[ht != [] and ht != "[]" for ht in tweet_df["hashtags"]]]
    return tagged_df

GetTaggedTweet(test_df)

,id,text,non_hashtags_text,author_id,hashtags
3,1512575894841434122,RT @LCSOfficial: From 5-time Challenger to the...,RT @<UNKTAG>Official: From 5-time Challenger t...,2297906262,[LCS]
19,1512575873613971458,RT @FOX5Vegas: #BTS doesn't kick off its Las V...,RT @FOX5Vegas: #<UNKTAG> doesn't kick off its ...,2482136157,[BTS]
20,1512575873089683464,RT @BelhaimarJamal: #IbroxStadium home of @Ran...,RT @BelhaimarJamal: #<UNKTAG> home of @<UNKTAG...,1299684274887634944,"[IbroxStadium, RangersFC, GlasgowRangers, Glas..."
24,1512575863266631680,"I’m selling Off my 2 #BTSTICKETS\nField A1, Ro...","I’m selling Off my 2 #<UNKTAG>\nField A1, Row ...",1411566745391976448,"[BTSTICKETS, BTSTICKETSELLING, PTD_ON_STAGE_LA..."
30,1512575854609518592,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: #<UNKTAG> mania has completely ...,1325256080272752641,[BTS]
31,1512575851883204608,RT @News3LV: #BTS has taken over the Las Vegas...,RT @News3LV: #<UNKTAG> has taken over the Las ...,3039980481,[BTS]
36,1512575841573744642,RT @brighttu0204: Still the best F4 intro..\n\...,RT @brighttu0204: Still the best F4 intro..\n\...,940934600255311872,[F4ThailandFinalEP]
37,1512575840860704770,"Round 🐰🐰🐰🐰🐰\nTomorrow, let’s have a great crow...","Round 🐰🐰🐰🐰🐰\nTomorrow, let’s have a great crow...",3100779943,"[ETR22, DTT22, BGS22]"
60,1512575797328224256,RT @FOX5Vegas: #BTS mania has completely taken...,RT @FOX5Vegas: #<UNKTAG> mania has completely ...,998193620233535488,[BTS]
68,1512575781611913223,@AppleTV @mlb @nationals I live 5 miles from t...,@AppleTV @mlb @nationals I live 5 miles from t...,355850575,[terrible]


In [39]:
# create the dataframe
df_list = []
tagged_df_list = []
queries_list = []
for i in range(10):
    queries_list.append(queries[100*i: 100*(i+1)])

for i in range(7, 10):
    print(f"Process the data for index = {i}")
    for query in tqdm(queries_list[i]):
        tweets = client.search_recent_tweets(query=query, 
                                             tweet_fields=['author_id', 'entities'],
                                             max_results=100)
        temp_df = CreateDataFrame(tweets)
        df_list.append(temp_df)
        tagged_df_list.append(GetTaggedTweet(temp_df))


    tweet_data = pd.concat(df_list)
    tagged_tweet_data = pd.concat(tagged_df_list)
    print(f"  Get a whole dataset of length {len(tweet_data)}")
    print(f"  Get a tagged dataset of length {len(tagged_tweet_data)}")
    print(f"  Output the data...")
    tweet_data.to_csv("./data/tweets_"+str(i)+".csv", index=False)
    tagged_tweet_data.to_csv("./data/tagged_tweets_"+str(i)+".csv", index=False)

Process the data for index = 7


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:50<00:00,  1.99it/s]


  Get a whole dataset of length 9999
  Get a tagged dataset of length 1487
  Output the data...
Process the data for index = 8


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:50<00:00,  1.97it/s]


  Get a whole dataset of length 19999
  Get a tagged dataset of length 3002
  Output the data...
Process the data for index = 9


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:50<00:00,  1.97it/s]


  Get a whole dataset of length 29999
  Get a tagged dataset of length 4631
  Output the data...


In [41]:
# read in data and concat them into one file
data_df_list = []
tagged_data_df_list = []
for i in range(10):
    tweet_data = pd.read_csv("./data/tweets_"+str(i)+".csv")
    tagged_tweet_data = pd.read_csv("./data/tagged_tweets_"+str(i)+".csv")
    data_df_list.append(tweet_data)
    tagged_data_df_list.append(tagged_tweet_data)

data_df = pd.concat(data_df_list)
tagged_data_df = pd.concat(tagged_data_df_list)
data_df.to_csv("./data/tweets.csv", index=False)
tagged_data_df.to_csv("./data/tagged_tweets.csv", index=False)

In [42]:
print(f"Get a whole dataset of length {len(data_df)}")
print(f"Get a tagged dataset of length {len(tagged_data_df)}")

Get a whole dataset of length 219989
Get a tagged dataset of length 34980
